## Задание

Данные берем отзывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи:

    1. построить свёрточные архитектуры
    2. построить различные архитектуры с RNN
    3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

---

In [2]:
!pip install --upgrade xlrd
!pip install pymorphy2
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.4 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 15.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=5cd0ebf8e50cf46aa0b5a729f216a13afd43f9bf819fb74c2ce81e1b74384bfa
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
import pandas as pd
import re

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

In [4]:
df = pd.read_excel('отзывы за лето.xls')

df.sample(15)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [5]:
df.Content = df.Content.astype(str)

In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [7]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
df['Content'] = df['Content'].apply(preprocess_text)

df.sample(5)

,Rating,Content,Date
6454,5,отличный приложение пользоваться минус мелочь,2017-08-04
13477,5,мненравиться,2017-07-25
6809,2,установка последний обновление непогружаться п...,2017-08-04
3080,5,👍,2017-08-09
17506,5,сбербанк отличный банк приложение мненравиться...,2017-07-11


In [9]:
train_df, test_val_df = train_test_split(df, test_size = 0.3, random_state = 49)
test_df, val_df = train_test_split(test_val_df, test_size = 0.5, random_state = 49)

In [10]:
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')
val_df.to_csv('val_df.csv')
test_val_df.to_csv('test_val_df.csv')

In [11]:
text_corpus_train = train_df['Content'].values
text_corpus_valid = val_df['Content'].values
text_corpus_test = test_df['Content'].values

In [12]:
num_classes = len(df['Rating'].unique())
num_classes

5

In [13]:
tokenizer = Tokenizer(num_words = None, 
                     filters = '#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes+1)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes+1)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes+1)

---

In [14]:
# RNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Masking(mask_value = 0.0))

model.add(SimpleRNN(132))
model.add(Dense(132, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))

model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 132, 30)           294870    
                                                                 
 masking (Masking)           (None, 132, 30)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 132)               21516     
                                                                 
 dense (Dense)               (None, 132)               17556     
                                                                 
 dropout (Dropout)           (None, 132)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 798       
                                                                 
Total params: 334,740
Trainable params: 334,740
Non-trai

In [15]:
history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 16s 545ms/step - loss: 1.0507 - accuracy: 0.6804 - val_loss: 0.7886 - val_accuracy: 0.7326
Epoch 2/10
26/26 [==============================] - 16s 605ms/step - loss: 0.8234 - accuracy: 0.7349 - val_loss: 0.7454 - val_accuracy: 0.7595
Epoch 3/10
26/26 [==============================] - 25s 980ms/step - loss: 0.7318 - accuracy: 0.7599 - val_loss: 0.6812 - val_accuracy: 0.7678
Epoch 4/10
26/26 [==============================] - 17s 640ms/step - loss: 0.6431 - accuracy: 0.7804 - val_loss: 0.6607 - val_accuracy: 0.7809
Epoch 5/10
26/26 [==============================] - 15s 580ms/step - loss: 0.5784 - accuracy: 0.7973 - val_loss: 0.6569 - val_accuracy: 0.7809
Epoch 6/10
26/26 [==============================] - 14s 533ms/step - loss: 0.5241 - accuracy: 0.8147 - val_loss: 0.6641 - val_accuracy: 0.7754
Epoch 7/10
26/26 [==============================] - 13s 516ms/step - loss: 0.4652 - accuracy: 0.8383 - val_loss: 0.6989 - val_accuracy: 0.7747

In [17]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 139ms/step - loss: 0.6870 - accuracy: 0.7715


Test score: 0.6869747042655945
Test accuracy: 0.7715392112731934


In [18]:
results = []

results.append(['RNN', score[0], score[1]])

---

In [19]:
# LSTM

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Masking(mask_value = 0.0))
model.add(LSTM(132, recurrent_dropout = 0.2))
model.add(Dense(132, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))

In [20]:
model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 132, 30)           294870    
                                                                 
 masking_1 (Masking)         (None, 132, 30)           0         
                                                                 
 lstm (LSTM)                 (None, 132)               86064     
                                                                 
 dense_2 (Dense)             (None, 132)               17556     
                                                                 
 dropout_1 (Dropout)         (None, 132)               0         
                                                                 
 dense_3 (Dense)             (None, 6)                 798       
                                                                 
Total params: 399,288
Trainable params: 399,288
Non-tr

In [21]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)  

history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 60s 2s/step - loss: 1.2078 - accuracy: 0.6839 - val_loss: 0.8779 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 53s 2s/step - loss: 0.8832 - accuracy: 0.7112 - val_loss: 0.7586 - val_accuracy: 0.7554
Epoch 3/10
26/26 [==============================] - 70s 3s/step - loss: 0.7607 - accuracy: 0.7503 - val_loss: 0.6858 - val_accuracy: 0.7699
Epoch 4/10
26/26 [==============================] - 68s 3s/step - loss: 0.6765 - accuracy: 0.7695 - val_loss: 0.6599 - val_accuracy: 0.7719
Epoch 5/10
26/26 [==============================] - 53s 2s/step - loss: 0.6205 - accuracy: 0.7839 - val_loss: 0.6517 - val_accuracy: 0.7844
Epoch 6/10
26/26 [==============================] - 53s 2s/step - loss: 0.5677 - accuracy: 0.7996 - val_loss: 0.6613 - val_accuracy: 0.7761
Epoch 7/10
26/26 [==============================] - 52s 2s/step - loss: 0.5403 - accuracy: 0.8098 - val_loss: 0.6752 - val_accuracy: 0.7706


In [22]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['LSTM', score[0], score[1]])

7/7 [==============================] - 3s 410ms/step - loss: 0.6737 - accuracy: 0.7709


Test score: 0.6737221479415894
Test accuracy: 0.7708938121795654


---

In [23]:
# GRU

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Masking(mask_value = 0.0))
model.add(GRU(64, recurrent_dropout = 0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))
model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 132, 30)           294870    
                                                                 
 masking_2 (Masking)         (None, 132, 30)           0         
                                                                 
 gru (GRU)                   (None, 64)                18432     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 390       
                                                                 
Total params: 317,852
Trainable params: 317,852
Non-tr

In [24]:
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 2, restore_best_weights = 1)  


history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 22s 737ms/step - loss: 1.4696 - accuracy: 0.6783 - val_loss: 0.9642 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 16s 634ms/step - loss: 0.9471 - accuracy: 0.7052 - val_loss: 0.7864 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 17s 662ms/step - loss: 0.8130 - accuracy: 0.7138 - val_loss: 0.7357 - val_accuracy: 0.7478
Epoch 4/10
26/26 [==============================] - 17s 639ms/step - loss: 0.7345 - accuracy: 0.7468 - val_loss: 0.7054 - val_accuracy: 0.7657
Epoch 5/10
26/26 [==============================] - 17s 652ms/step - loss: 0.6681 - accuracy: 0.7746 - val_loss: 0.6851 - val_accuracy: 0.7671
Epoch 6/10
26/26 [==============================] - 17s 648ms/step - loss: 0.6106 - accuracy: 0.7911 - val_loss: 0.6765 - val_accuracy: 0.7692
Epoch 7/10
26/26 [==============================] - 17s 660ms/step - loss: 0.5607 - accuracy: 0.8055 - val_loss: 0.6898 - val_accuracy: 0.7685

In [25]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['GRU', score[0], score[1]])

7/7 [==============================] - 1s 110ms/step - loss: 0.6928 - accuracy: 0.7599


Test score: 0.6928486227989197
Test accuracy: 0.7599225640296936


---

In [26]:
# CNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))

model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 132, 30)           294870    
                                                                 
 conv1d (Conv1D)             (None, 130, 128)          11648     
                                                                 
 activation (Activation)     (None, 130, 128)          0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_6 (Dense)             (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                      

In [27]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)  


history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 5s 175ms/step - loss: 1.6141 - accuracy: 0.6700 - val_loss: 1.2803 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 4s 167ms/step - loss: 1.2350 - accuracy: 0.7038 - val_loss: 0.9323 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 4s 166ms/step - loss: 1.0858 - accuracy: 0.7054 - val_loss: 0.8059 - val_accuracy: 0.7422
Epoch 4/10
26/26 [==============================] - 5s 174ms/step - loss: 0.9526 - accuracy: 0.7200 - val_loss: 0.7047 - val_accuracy: 0.7713
Epoch 5/10
26/26 [==============================] - 4s 171ms/step - loss: 0.8620 - accuracy: 0.7307 - val_loss: 0.6760 - val_accuracy: 0.7802
Epoch 6/10
26/26 [==============================] - 5s 175ms/step - loss: 0.7992 - accuracy: 0.7447 - val_loss: 0.6666 - val_accuracy: 0.7816
Epoch 7/10
26/26 [==============================] - 4s 172ms/step - loss: 0.7733 - accuracy: 0.7499 - val_loss: 0.6590 - val_accuracy: 0.7830
Epoch 

In [28]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['CNN', score[0], score[1]])

7/7 [==============================] - 0s 40ms/step - loss: 0.6996 - accuracy: 0.7722


Test score: 0.6996115446090698
Test accuracy: 0.7721845507621765


---

In [29]:
# RNN + CNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(SimpleRNN(132, recurrent_dropout = 0.2, return_sequences = "True"))
model.add(Conv1D(132, 3, activation = "linear"))
model.add(Conv1D(64, 1, activation = "linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes+1, activation = "softmax"))      


model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 132, 30)           294870    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 132, 132)          21516     
                                                                 
 conv1d_1 (Conv1D)           (None, 130, 132)          52404     
                                                                 
 conv1d_2 (Conv1D)           (None, 130, 64)           8512      
                                                                 
 flatten (Flatten)           (None, 8320)              0         
                                                                 
 dropout_4 (Dropout)         (None, 8320)              0         
                                                                 
 dense_8 (Dense)             (None, 6)                

In [30]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)  


history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 35s 1s/step - loss: 1.1316 - accuracy: 0.6923 - val_loss: 0.7953 - val_accuracy: 0.7512
Epoch 2/10
26/26 [==============================] - 32s 1s/step - loss: 0.8042 - accuracy: 0.7362 - val_loss: 0.7488 - val_accuracy: 0.7491
Epoch 3/10
26/26 [==============================] - 32s 1s/step - loss: 0.6834 - accuracy: 0.7679 - val_loss: 0.6750 - val_accuracy: 0.7816
Epoch 4/10
26/26 [==============================] - 32s 1s/step - loss: 0.5959 - accuracy: 0.7890 - val_loss: 0.6820 - val_accuracy: 0.7795
Epoch 5/10
26/26 [==============================] - 32s 1s/step - loss: 0.5347 - accuracy: 0.8047 - val_loss: 0.7286 - val_accuracy: 0.7795


In [31]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['RNN + CNN', score[0], score[1]])

7/7 [==============================] - 2s 274ms/step - loss: 0.7066 - accuracy: 0.7554


Test score: 0.7065978646278381
Test accuracy: 0.7554049491882324


---

In [32]:
results_df = pd.DataFrame(results, columns = ['Model', 'Test score', 'Test accuracy'])
results_df

,Model,Test score,Test accuracy
0,RNN,0.686975,0.771539
1,LSTM,0.673722,0.770894
2,GRU,0.692849,0.759923
3,CNN,0.699612,0.772185
4,RNN + CNN,0.706598,0.755405
